Technological Institute of the Philippines | Quezon City - Computer Engineering
--- | ---
Course Code: | CPE 019
Code Title: | Emerging Technologies in CpE 2
2nd Semester | AY 2023-2024
<hr> | <hr>
**Assignment 7.1:**| Classifications and Regression
<hr> | <hr>
**Name**         | Naga, Jamal
**Section** | CPE32S3
**Date Performed**: |April 3, 2024
**Date Submitted**: |April 11, 2024
**Instructor**: | Engr. Roman Richard
<hr> | <hr>

# **Instructions:**
1. Choose any dataset applicable to the classification problem, and also, choose any dataset applicable to the regression problem.
2. Explain your datasets and the problem being addressed. <br>
3. For classification, do the following:<br>
- Create a base model
- Evaluate the model with k-fold cross validation
- Improve the accuracy of your model by applying additional hidden layers <br>
4. For regression, do the following:<br>
-  Create a base model
- Improve the model by standardizing the dataset
- Show tuning of layers and neurons (see evaluating small and larger networks)


# Classification Problem

## **Dataset exlaination and problem being addressed**

## 1. **Dataset:** Breast Cancer Wisconsin (Diagnostic) - This dataset contains various features computed from images of breast masses, such as radius, texture, perimeter, area, smoothness, compactness, concavity, concave points, symmetry, and fractal dimension. The target variable is the diagnosis, which can be either "M" for malignant or "B" for benign.
https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic
## 2. The problem being addressed by this dataset is the classification of breast masses as either benign or malignant based on the computed features. This is a crucial problem in the field of medical diagnostics, as accurately identifying malignant masses can aid in timely treatment and improve patient outcomes.

In [ ]:
pip install ucimlrepo

In [ ]:
# Load the necessary libraries
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras import layers,models
from sklearn.model_selection import KFold
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
## Import Keras objects for Deep Learning

from keras.models  import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

# metadata
print(breast_cancer_wisconsin_diagnostic.metadata)

# variable information
print(breast_cancer_wisconsin_diagnostic.variables)


{'uci_id': 17, 'name': 'Breast Cancer Wisconsin (Diagnostic)', 'repository_url': 'https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic', 'data_url': 'https://archive.ics.uci.edu/static/public/17/data.csv', 'abstract': 'Diagnostic Wisconsin Breast Cancer Database.', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 569, 'num_features': 30, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Diagnosis'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1993, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5DW2B', 'creators': ['William Wolberg', 'Olvi Mangasarian', 'Nick Street', 'W. Street'], 'intro_paper': {'title': 'Nuclear feature extraction for breast tumor diagnosis', 'authors': 'W. Street, W. Wolberg, O. Mangasarian', 'published_in': 'Electronic imaging', 'year': 1993, 'url': 'https://www.semanticscholar.org/paper/53

# Preprocessing

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   radius1             569 non-null    float64
 1   texture1            569 non-null    float64
 2   perimeter1          569 non-null    float64
 3   area1               569 non-null    float64
 4   smoothness1         569 non-null    float64
 5   compactness1        569 non-null    float64
 6   concavity1          569 non-null    float64
 7   concave_points1     569 non-null    float64
 8   symmetry1           569 non-null    float64
 9   fractal_dimension1  569 non-null    float64
 10  radius2             569 non-null    float64
 11  texture2            569 non-null    float64
 12  perimeter2          569 non-null    float64
 13  area2               569 non-null    float64
 14  smoothness2         569 non-null    float64
 15  compactness2        569 non-null    float64
 16  concavit

In [ ]:
X.head()

,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,fractal_dimension1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
y.head()

,Diagnosis
0,M
1,M
2,M
3,M
4,M


In [ ]:
y['Diagnosis'] = y['Diagnosis'].replace({'M': 1, 'B': 0})
y['Diagnosis'] = y['Diagnosis'].astype('int64')

**Observation**: Upon inspecting the values and data type of the 'y' variable and observing the categorical values 'M' and 'B' in the 'Diagnosis' column, I replaced them with numerical equivalents (1 and 0 respectively), and I converted the column into int64.

In [ ]:
y.head()

,Diagnosis
0,1
1,1
2,1
3,1
4,1


In [ ]:
print((X.shape))
print((y.shape))

(569, 30)
(569, 1)


In [ ]:
# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=11111)

In [ ]:
# Reset DataFrame indices
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

**Observation:** Since I continuously encountered errors due to misaligned indices, I reset them to ensure that each row in the feature DataFrame corresponds correctly to its corresponding target value in the target DataFrame. This ensures proper alignment during model training, preventing any potential issues that may arise from misaligned indices and ensuring the smooth execution of subsequent operations.

## Base Model

In [ ]:
# Define the number of folds for cross-validation
k = 5  # Example: 5-fold cross-validation
cv = KFold(n_splits=k, shuffle=True, random_state=11111)

# Initialize lists to store evaluation results
accuracies = []
losses = []

# Base Model Architecture
base_model_architecture = [
    Dense(23, input_shape=(30,), activation="relu"),
    Dense(1, activation='relu'),  # Base Model: Single hidden layer with 1 neuron and ReLU activation
    Dropout(rate=0.5),
    Dense(1, activation="sigmoid")
]

# Perform k-fold cross-validation
for fold, (train_index, val_index) in enumerate(cv.split(X_train, y_train), 1):
    # Split data into training and validation sets
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Create a new model instance
    model = Sequential(base_model_architecture)

    # Choose optimizer and compile the model
    optimizer = Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

    # Train the model
    history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), epochs=10, verbose=0)

    # Print loss and accuracy for each epoch
    for epoch in range(len(history.history['loss'])):
        print(f"Fold: {fold}, Epoch: {epoch+1}, Loss: {history.history['loss'][epoch]}, Accuracy: {history.history['accuracy'][epoch]}")

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    losses.append(loss)
    accuracies.append(accuracy)

# Calculate the mean and standard deviation of validation metrics
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
mean_loss = np.mean(losses)
std_loss = np.std(losses)

Fold: 1, Epoch: 1, Loss: 0.6926031708717346, Accuracy: 0.6147058606147766
Fold: 1, Epoch: 2, Loss: 0.6914857029914856, Accuracy: 0.6147058606147766
Fold: 1, Epoch: 3, Loss: 0.6904550790786743, Accuracy: 0.6147058606147766
Fold: 1, Epoch: 4, Loss: 0.689439594745636, Accuracy: 0.6147058606147766
Fold: 1, Epoch: 5, Loss: 0.6884393692016602, Accuracy: 0.6147058606147766
Fold: 1, Epoch: 6, Loss: 0.6876000165939331, Accuracy: 0.6147058606147766
Fold: 1, Epoch: 7, Loss: 0.6865940690040588, Accuracy: 0.6147058606147766
Fold: 1, Epoch: 8, Loss: 0.6858449578285217, Accuracy: 0.6147058606147766
Fold: 1, Epoch: 9, Loss: 0.6850900650024414, Accuracy: 0.6147058606147766
Fold: 1, Epoch: 10, Loss: 0.6843073964118958, Accuracy: 0.6147058606147766


Fold: 2, Epoch: 1, Loss: 0.6847102046012878, Accuracy: 0.6011730432510376
Fold: 2, Epoch: 2, Loss: 0.6840406060218811, Accuracy: 0.6011730432510376
Fold: 2, Epoch: 3, Loss: 0.6835286617279053, Accuracy: 0.6011730432510376
Fold: 2, Epoch: 4, Loss: 0.6829640865325928, Accuracy: 0.6011730432510376
Fold: 2, Epoch: 5, Loss: 0.6824377775192261, Accuracy: 0.6011730432510376
Fold: 2, Epoch: 6, Loss: 0.6819186806678772, Accuracy: 0.6011730432510376
Fold: 2, Epoch: 7, Loss: 0.6814655661582947, Accuracy: 0.6011730432510376
Fold: 2, Epoch: 8, Loss: 0.6810013651847839, Accuracy: 0.6011730432510376
Fold: 2, Epoch: 9, Loss: 0.680578351020813, Accuracy: 0.6011730432510376
Fold: 2, Epoch: 10, Loss: 0.6802029609680176, Accuracy: 0.6011730432510376


Fold: 3, Epoch: 1, Loss: 0.6802423000335693, Accuracy: 0.5982404947280884
Fold: 3, Epoch: 2, Loss: 0.6799454689025879, Accuracy: 0.5982404947280884
Fold: 3, Epoch: 3, Loss: 0.6796570420265198, Accuracy: 0.5982404947280884
Fold: 3, Epoch: 4, Loss: 0.679353654384613, Accuracy: 0.5982404947280884
Fold: 3, Epoch: 5, Loss: 0.6791196465492249, Accuracy: 0.5982404947280884
Fold: 3, Epoch: 6, Loss: 0.6788846254348755, Accuracy: 0.5982404947280884
Fold: 3, Epoch: 7, Loss: 0.6785945296287537, Accuracy: 0.5982404947280884
Fold: 3, Epoch: 8, Loss: 0.6783769726753235, Accuracy: 0.5982404947280884
Fold: 3, Epoch: 9, Loss: 0.6781706809997559, Accuracy: 0.5982404947280884
Fold: 3, Epoch: 10, Loss: 0.6779063940048218, Accuracy: 0.5982404947280884


Fold: 4, Epoch: 1, Loss: 0.6771465539932251, Accuracy: 0.6011730432510376
Fold: 4, Epoch: 2, Loss: 0.6768959760665894, Accuracy: 0.6011730432510376
Fold: 4, Epoch: 3, Loss: 0.6767082214355469, Accuracy: 0.6011730432510376
Fold: 4, Epoch: 4, Loss: 0.676445722579956, Accuracy: 0.6011730432510376
Fold: 4, Epoch: 5, Loss: 0.6763089299201965, Accuracy: 0.6011730432510376
Fold: 4, Epoch: 6, Loss: 0.6760225892066956, Accuracy: 0.6011730432510376
Fold: 4, Epoch: 7, Loss: 0.6758122444152832, Accuracy: 0.6011730432510376
Fold: 4, Epoch: 8, Loss: 0.6756319999694824, Accuracy: 0.6011730432510376
Fold: 4, Epoch: 9, Loss: 0.6754252910614014, Accuracy: 0.6011730432510376
Fold: 4, Epoch: 10, Loss: 0.6753501296043396, Accuracy: 0.6011730432510376
Fold: 5, Epoch: 1, Loss: 0.6720541715621948, Accuracy: 0.6129032373428345
Fold: 5, Epoch: 2, Loss: 0.6717135906219482, Accuracy: 0.6129032373428345
Fold: 5, Epoch: 3, Loss: 0.6715281009674072, Accuracy: 0.6129032373428345
Fold: 5, Epoch: 4, Loss: 0.67128217220

In [ ]:
# Calculate the mean and standard deviation of validation metrics
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
mean_loss = np.mean(losses)
std_loss = np.std(losses)

print("Mean Accuracy (Base Model):", mean_accuracy)
print("Standard Deviation of Accuracy:", std_accuracy)
print("Mean Loss:", mean_loss)
print("Standard Deviation of Loss:", std_loss)

Mean Accuracy (Base Model): 0.6057182073593139
Standard Deviation of Accuracy: 0.02704476489084705
Mean Loss: 0.6769299983978272
Standard Deviation of Loss: 0.0070508964722141


## Improved Model

In [ ]:
# Define the number of folds for cross-validation
k = 5  # Example: 5-fold cross-validation
cv = KFold(n_splits=k, shuffle=True, random_state=11111)

# Initialize lists to store evaluation results
accuracies = []
losses = []

# Base Model Architecture (Improved Model)
improved_model_architecture = [
    Dense(64, input_shape=(30,), activation="relu"),  # Additional hidden layer with 64 neurons
    Dense(32, activation='relu'),  # Additional hidden layer with 32 neurons
    Dropout(rate=0.5),  # Dropout layer
    Dense(16, activation='relu'),  # Additional hidden layer with 16 neurons
    Dense(1, activation="sigmoid")  # Output layer
]

# Perform k-fold cross-validation
for fold, (train_index, val_index) in enumerate(cv.split(X_train, y_train), 1):
    # Split data into training and validation sets
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Create a new model instance
    model = Sequential(improved_model_architecture)

    # Choose optimizer and compile the model
    optimizer = Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

    # Train the model
    history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), epochs=10, verbose=0)

    # Print loss and accuracy for each epoch
    for epoch in range(len(history.history['loss'])):
        print(f"Fold: {fold}, Epoch: {epoch+1}, Loss: {history.history['loss'][epoch]}, Accuracy: {history.history['accuracy'][epoch]}")

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    losses.append(loss)
    accuracies.append(accuracy)

# Calculate the mean and standard deviation of validation metrics
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
mean_loss = np.mean(losses)
std_loss = np.std(losses)

Fold: 1, Epoch: 1, Loss: 16.475759506225586, Accuracy: 0.5411764979362488
Fold: 1, Epoch: 2, Loss: 8.255961418151855, Accuracy: 0.5588235259056091
Fold: 1, Epoch: 3, Loss: 3.3842318058013916, Accuracy: 0.6617646813392639
Fold: 1, Epoch: 4, Loss: 1.6184890270233154, Accuracy: 0.6764705777168274
Fold: 1, Epoch: 5, Loss: 1.0037661790847778, Accuracy: 0.7411764860153198
Fold: 1, Epoch: 6, Loss: 0.5702606439590454, Accuracy: 0.7382352948188782
Fold: 1, Epoch: 7, Loss: 0.42970335483551025, Accuracy: 0.8058823347091675
Fold: 1, Epoch: 8, Loss: 0.4221045672893524, Accuracy: 0.8500000238418579
Fold: 1, Epoch: 9, Loss: 0.42539000511169434, Accuracy: 0.8470588326454163
Fold: 1, Epoch: 10, Loss: 0.338969886302948, Accuracy: 0.8588235378265381


Fold: 2, Epoch: 1, Loss: 0.39901402592658997, Accuracy: 0.8563050031661987
Fold: 2, Epoch: 2, Loss: 0.3990137577056885, Accuracy: 0.8357771039009094
Fold: 2, Epoch: 3, Loss: 0.3864862024784088, Accuracy: 0.8651026487350464
Fold: 2, Epoch: 4, Loss: 0.3683061897754669, Accuracy: 0.8533724546432495
Fold: 2, Epoch: 5, Loss: 0.35351067781448364, Accuracy: 0.8387096524238586
Fold: 2, Epoch: 6, Loss: 0.33724647760391235, Accuracy: 0.8563050031661987
Fold: 2, Epoch: 7, Loss: 0.3878612816333771, Accuracy: 0.8475073575973511
Fold: 2, Epoch: 8, Loss: 0.36341944336891174, Accuracy: 0.8621701002120972
Fold: 2, Epoch: 9, Loss: 0.3550839126110077, Accuracy: 0.8651026487350464
Fold: 2, Epoch: 10, Loss: 0.32009634375572205, Accuracy: 0.8445748090744019


Fold: 3, Epoch: 1, Loss: 0.411895215511322, Accuracy: 0.8240469098091125
Fold: 3, Epoch: 2, Loss: 0.3710653483867645, Accuracy: 0.8651026487350464
Fold: 3, Epoch: 3, Loss: 0.318603515625, Accuracy: 0.8533724546432495
Fold: 3, Epoch: 4, Loss: 0.3522436022758484, Accuracy: 0.8445748090744019
Fold: 3, Epoch: 5, Loss: 0.28642529249191284, Accuracy: 0.8797653913497925
Fold: 3, Epoch: 6, Loss: 0.3283699154853821, Accuracy: 0.8533724546432495
Fold: 3, Epoch: 7, Loss: 0.33006390929222107, Accuracy: 0.8387096524238586
Fold: 3, Epoch: 8, Loss: 0.3404671251773834, Accuracy: 0.8445748090744019
Fold: 3, Epoch: 9, Loss: 0.35359132289886475, Accuracy: 0.8240469098091125
Fold: 3, Epoch: 10, Loss: 0.36763983964920044, Accuracy: 0.8328445553779602


Fold: 4, Epoch: 1, Loss: 0.33210688829421997, Accuracy: 0.8826979398727417
Fold: 4, Epoch: 2, Loss: 0.28471747040748596, Accuracy: 0.8621701002120972
Fold: 4, Epoch: 3, Loss: 0.36882898211479187, Accuracy: 0.829912006855011
Fold: 4, Epoch: 4, Loss: 0.3099607229232788, Accuracy: 0.8651026487350464
Fold: 4, Epoch: 5, Loss: 0.32314303517341614, Accuracy: 0.8826979398727417
Fold: 4, Epoch: 6, Loss: 0.3380014896392822, Accuracy: 0.8651026487350464
Fold: 4, Epoch: 7, Loss: 0.2720433473587036, Accuracy: 0.8856304883956909
Fold: 4, Epoch: 8, Loss: 0.2789344787597656, Accuracy: 0.8885630369186401
Fold: 4, Epoch: 9, Loss: 0.32609471678733826, Accuracy: 0.8885630369186401
Fold: 4, Epoch: 10, Loss: 0.3292766809463501, Accuracy: 0.8651026487350464
Fold: 5, Epoch: 1, Loss: 0.3485739827156067, Accuracy: 0.8504399061203003
Fold: 5, Epoch: 2, Loss: 0.35783651471138, Accuracy: 0.8504399061203003
Fold: 5, Epoch: 3, Loss: 0.31964385509490967, Accuracy: 0.8416422009468079
Fold: 5, Epoch: 4, Loss: 0.3053539

In [ ]:
print("Mean Accuracy (Improved Model):", mean_accuracy)
print("Standard Deviation of Accuracy:", std_accuracy)
print("Mean Loss:", mean_loss)
print("Standard Deviation of Loss:", std_loss)

Mean Accuracy (Improved Model): 0.9060465216636657
Standard Deviation of Accuracy: 0.01687156473463425
Mean Loss: 0.2455670267343521
Standard Deviation of Loss: 0.055985485886159515


**Evaluation:**
The comparison between the base model and the improved model reveals a substantial enhancement in performance metrics. Specifically, the improved model achieves a significantly higher mean accuracy of 90.6% compared to the base model's 60.6%, indicating a marked improvement in distinguishing between benign and malignant breast masses. Moreover, the improved model exhibits lower mean loss and standard deviation of loss, reflecting a more precise prediction of class probabilities and increased stability across different folds of the cross-validation process.

# Regression Problem

## **Dataset exlaination and problem being addressed**

## 1. **Dataset:** Concrete Compressive Strength - The Concrete Compressive Strength dataset contains features like cement, water, and aggregates, along with compressive strength, used to predict concrete strength, aiding engineers in optimizing mixtures for various applications.
https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength
## 2. This dataset addresses the prediction of concrete compressive strength, vital for optimizing mixtures to meet strength requirements, thus improving construction efficiency and structural durability.

In [ ]:
# Load the necessary libraries
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras import layers,models
from sklearn.model_selection import KFold
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
## Import Keras objects for Deep Learning

from keras.models  import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
concrete_compressive_strength = fetch_ucirepo(id=165)

# data (as pandas dataframes)
X = concrete_compressive_strength.data.features
y = concrete_compressive_strength.data.targets

# metadata
print(concrete_compressive_strength.metadata)

# variable information
print(concrete_compressive_strength.variables)


{'uci_id': 165, 'name': 'Concrete Compressive Strength', 'repository_url': 'https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength', 'data_url': 'https://archive.ics.uci.edu/static/public/165/data.csv', 'abstract': 'Concrete is the most important material in civil engineering. The concrete compressive strength is a highly nonlinear function of age and ingredients. ', 'area': 'Physics and Chemistry', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 1030, 'num_features': 8, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Concrete compressive strength'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1998, 'last_updated': 'Sun Feb 11 2024', 'dataset_doi': '10.24432/C5PK67', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'title': 'Modeling of strength of high-performance concrete using artificial neural networks', 'authors': 'I. Yeh', 'published_in': 'Cement and Concrete Res

# Preprocessing

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cement              1030 non-null   float64
 1   Blast Furnace Slag  1030 non-null   float64
 2   Fly Ash             1030 non-null   float64
 3   Water               1030 non-null   float64
 4   Superplasticizer    1030 non-null   float64
 5   Coarse Aggregate    1030 non-null   float64
 6   Fine Aggregate      1030 non-null   float64
 7   Age                 1030 non-null   int64  
dtypes: float64(7), int64(1)
memory usage: 64.5 KB


In [ ]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 1 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Concrete compressive strength  1030 non-null   float64
dtypes: float64(1)
memory usage: 8.2 KB


In [ ]:
X.head(10)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360
5,266.0,114.0,0.0,228.0,0.0,932.0,670.0,90
6,380.0,95.0,0.0,228.0,0.0,932.0,594.0,365
7,380.0,95.0,0.0,228.0,0.0,932.0,594.0,28
8,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28
9,475.0,0.0,0.0,228.0,0.0,932.0,594.0,28


In [ ]:
y.head()

,Concrete compressive strength
0,79.99
1,61.89
2,40.27
3,41.05
4,44.30


In [ ]:
X.shape

(1030, 8)

In [ ]:
# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Standardized the data

In [ ]:
# Scaling the Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Define base model network architecture
base_model = Sequential([
    Dense(11, input_shape=(8,), activation="relu"),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation = 'linear')  # No activation function for regression task (linear activation is used by default)
])

base_model.compile(Adam(learning_rate = .01), "mean_squared_error", metrics=["mae"])

# Train the model
run_hist_1 = base_model.fit(X_train_scaled, y_train, validation_data=(X_test, y_test), epochs=50)

Epoch 1/50
26/26 [==============================] - 1s 10ms/step - loss: 1273.8286 - mae: 31.2312 - val_loss: 35599276.0000 - val_mae: 5918.5933
Epoch 2/50
26/26 [==============================] - 0s 4ms/step - loss: 374.8046 - mae: 15.2387 - val_loss: 59335704.0000 - val_mae: 7670.4106
Epoch 3/50
26/26 [==============================] - 0s 3ms/step - loss: 192.7343 - mae: 11.2066 - val_loss: 64057052.0000 - val_mae: 7973.7744
Epoch 4/50
26/26 [==============================] - 0s 4ms/step - loss: 169.1156 - mae: 10.5937 - val_loss: 69188592.0000 - val_mae: 8288.2695
Epoch 5/50
26/26 [==============================] - 0s 4ms/step - loss: 150.8590 - mae: 10.0327 - val_loss: 73453104.0000 - val_mae: 8542.9258
Epoch 6/50
26/26 [==============================] - 0s 3ms/step - loss: 141.9506 - mae: 9.7358 - val_loss: 75148112.0000 - val_mae: 8641.9609
Epoch 7/50
26/26 [==============================] - 0s 3ms/step - loss: 136.8158 - mae: 9.5146 - val_loss: 79529160.0000 - val_mae: 8893.5107

In [ ]:
# Evaluate base model
base_train_loss, base_train_mae = base_model.evaluate(X_train_scaled, y_train, verbose=0)
base_test_loss, base_test_mae = base_model.evaluate(X_test_scaled, y_test, verbose=0)

print("Larger Model Performance:")
print("Train Loss:", base_train_loss)
print("Train MAE:", base_train_mae)
print("Test Loss:", base_test_loss)
print("Test MAE:", base_test_mae)

Larger Model Performance:
Train Loss: 37.785892486572266
Train MAE: 4.774435520172119
Test Loss: 32.04681396484375
Test MAE: 4.366782188415527


Observation: The base model consists of 11 neurons in the first layer, 8 neurons in the second layer, and 4 neurons in the third layer. The observed higher test error compared to the training error suggests potential overfitting, likely due to the model's limited capacity to capture the complexity of the underlying data. This limitation may arise from the relatively small number of neurons in each layer, hindering the model's ability to learn intricate patterns and generalize effectively to unseen data.








# Tuning of layers and neurons of small and larger networks

In [ ]:
# Define smaller network architecture
small_model = Sequential([
    Dense(12, input_shape=(8,), activation="relu"),
    Dense(1, activation='relu'),
    Dense(1, activation = 'linear')
])

small_model.compile(Adam(learning_rate = .01), "mean_squared_error", metrics=["mae"])

In [ ]:
# Train the model
# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
run_hist_1 = small_model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=50)

Epoch 1/50
26/26 [==============================] - 1s 9ms/step - loss: 1286.4071 - mae: 32.1612 - val_loss: 991.3027 - val_mae: 27.7660
Epoch 2/50
26/26 [==============================] - 0s 3ms/step - loss: 778.7847 - mae: 23.7842 - val_loss: 507.7591 - val_mae: 18.7048
Epoch 3/50
26/26 [==============================] - 0s 3ms/step - loss: 362.1989 - mae: 15.1304 - val_loss: 257.2556 - val_mae: 13.0590
Epoch 4/50
26/26 [==============================] - 0s 4ms/step - loss: 219.4953 - mae: 11.6932 - val_loss: 192.2598 - val_mae: 11.5681
Epoch 5/50
26/26 [==============================] - 0s 4ms/step - loss: 178.3139 - mae: 10.6468 - val_loss: 164.8468 - val_mae: 10.7789
Epoch 6/50
26/26 [==============================] - 0s 3ms/step - loss: 157.5621 - mae: 10.1517 - val_loss: 148.8991 - val_mae: 10.2295
Epoch 7/50
26/26 [==============================] - 0s 3ms/step - loss: 143.8277 - mae: 9.7523 - val_loss: 135.0804 - val_mae: 9.7288
Epoch 8/50
26/26 [==============================]

In [ ]:
# Evaluate smaller model
small_train_loss, small_train_mae = small_model.evaluate(X_train_scaled, y_train, verbose=0)
small_test_loss, small_test_mae = small_model.evaluate(X_test_scaled, y_test, verbose=0)

print("Smaller Model Performance:")
print("Train Loss:", small_train_loss)
print("Train MAE:", small_train_mae)
print("Test Loss:", small_test_loss)
print("Test MAE:", small_test_mae)

Smaller Model Performance:
Train Loss: 39.10651779174805
Train MAE: 4.81760311126709
Test Loss: 40.92934036254883
Test MAE: 4.737294673919678


In [ ]:
# Define larger network architecture
large_model = Sequential([
    Dense(128, input_shape=(8,), activation="relu"),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='linear')
])
large_model.compile(Adam(learning_rate = .01), "mean_squared_error", metrics=["mae"])

In [ ]:
# Train the model
# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
run_hist_1 = large_model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=50)

Epoch 1/50
26/26 [==============================] - 0s 11ms/step - loss: 24.9064 - mae: 3.7399 - val_loss: 29.5732 - val_mae: 4.1266
Epoch 2/50
26/26 [==============================] - 0s 10ms/step - loss: 22.6819 - mae: 3.6046 - val_loss: 41.0424 - val_mae: 4.9175
Epoch 3/50
26/26 [==============================] - 0s 7ms/step - loss: 32.9291 - mae: 4.3383 - val_loss: 41.3989 - val_mae: 5.0458
Epoch 4/50
26/26 [==============================] - 0s 7ms/step - loss: 27.3385 - mae: 4.0572 - val_loss: 36.5406 - val_mae: 4.5210
Epoch 5/50
26/26 [==============================] - 0s 7ms/step - loss: 27.8486 - mae: 4.0102 - val_loss: 34.7226 - val_mae: 4.5461
Epoch 6/50
26/26 [==============================] - 0s 5ms/step - loss: 24.2567 - mae: 3.7965 - val_loss: 35.5899 - val_mae: 4.5806
Epoch 7/50
26/26 [==============================] - 0s 5ms/step - loss: 22.3952 - mae: 3.6144 - val_loss: 32.0777 - val_mae: 4.2137
Epoch 8/50
26/26 [==============================] - 0s 5ms/step - loss: 20

In [ ]:
# Evaluate larger model
large_train_loss, large_train_mae = large_model.evaluate(X_train_scaled, y_train, verbose=0)
large_test_loss, large_test_mae = large_model.evaluate(X_test_scaled, y_test, verbose=0)

print("Larger Model Performance:")
print("Train Loss:", large_train_loss)
print("Train MAE:", large_train_mae)
print("Test Loss:", large_test_loss)
print("Test MAE:", large_test_mae)

Larger Model Performance:
Train Loss: 12.103827476501465
Train MAE: 2.693608522415161
Test Loss: 28.16518783569336
Test MAE: 3.845824718475342


**Evaluation**: The base model, with its simple structure and a moderate number of neurons, performed decently, but it could benefit from expanding its neural network architecture for improved performance. However, the small model, which doesn't have many neurons, struggled, showing it's important to have enough capacity in the model. On the other hand, the larger model, which has more layers and neurons, did much better, proving that adjusting these factors can really help the model understand complex patterns in the data and make better predictions.


# **Conclusion**:

For the classification problem, I utilized the "Breast Cancer Wisconsin (Diagnostic) Data Set" to predict malignancy in breast masses. After constructing a base classification model, I enhanced its accuracy using k-fold cross-validation and additional hidden layers in the neural network architecture.

For the regression model trained on the Concrete Compressive Strength dataset effectively predicts concrete compressive strength based on its mixture composition. The model's performance metrics, including low training and testing losses, as well as mean absolute error, demonstrate its ability to accurately estimate concrete strength, facilitating optimization of concrete mixtures for various construction applications.

In conclusion, these experiments showcase the versatility of neural networks in tackling both classification and regression tasks. By refining model architectures and leveraging appropriate preprocessing techniques, we can achieve significant improvements in predictive accuracy across different domains.